In [69]:
#To check for precedents
import json
with open('jsonformatter.txt', 'r') as f:
    with open("Precedents output.txt", 'w') as out:
        data = json.load(f)

        # print(type(data['label']))
        for case in data:
            out.write('Case id : ' + str(case['id']) + '\n')
            annotation = case['annotations'][0]
            # print(annotation['result'])
            for key in annotation['result']:
                # out.write(key['value']['labels'][0] + '\n')
                if key['value']['labels'][0] == 'STA':
                    out.write('STA : ' + key['value']['text'] + '\n\n')

            out.write("\n")

In [3]:
#To check for precedents relied
import json
with open('jsonformatter.txt', 'r') as f:
    with open("Precedents relied output.txt", 'w') as out:
        data = json.load(f)

        for case in data:
            out.write('Id : ' + str(case['id']) + '\n')
            annotation = case['annotations'][0]
            for key in annotation['result']:
                if key['value']['labels'][0] == 'PRE_RELIED':
                    out.write('PRE_RELIED : ' + key['value']['text'] + '\n\n')

            out.write("\n")

In [4]:
#To check for precedents not relied
import json
with open('jsonformatter.txt', 'r') as f:
    with open("Precedents not relied output.txt", 'w') as out:
        data = json.load(f)

        for case in data:
            out.write('Id : ' + str(case['id']) + '\n')
            annotation = case['annotations'][0]
            for key in annotation['result']:
                if key['value']['labels'][0] == 'PRE_NOT_RELIED':
                    out.write('PRE_NOT_RELIED : ' + key['value']['text'] + '\n\n')

            out.write("\n")

In [25]:
#To seperate the precedents on the keyword "vs" or "v."

import xml.etree.ElementTree as ET
import os

directory = 'Writ Petition'
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    mytree = ET.ElementTree(file= f)
    myroot = mytree.getroot()

    out_dir = directory+' precedents output'
    out_file = os.path.join(out_dir, filename.replace(".xml", " output.txt"))
    with open( out_file, 'w') as out:
        for i in myroot:
            if(i.tag=='JudgmentText'):
                for legis in i:
                    # out.write("Inside Judgement Text : " + legis.tag + '\n')
                    if(legis.tag=='I'):
                        for para in legis:
                            if para.text is None:
                                continue
                            s = para.text.lower()
                            if 'vs' in s or ' v. ' in s:  # """'in the case' in s or 'that case' in s or""" 
                                out.write("Precedent : " + para.text + '\n\n')

                    elif(legis.tag=='P'):

                        if legis.text is None:
                            continue

                        s = legis.text.lower()
                        if 'vs' in s or ' v. ' in s:  # """'in the case' in s or 'that case' in s or""" 
                            out.write("Precedent : " + legis.text + '\n\n')